In [121]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
import pickle

In [122]:
df=pd.read_csv('./Datasets/Training.csv')

In [123]:
df.sample(5)

,itching,skin_rash,nodal_skin_eruptions,continuous_sneezing,shivering,chills,joint_pain,stomach_pain,acidity,ulcers_on_tongue,...,blackheads,scurring,skin_peeling,silver_like_dusting,small_dents_in_nails,inflammatory_nails,blister,red_sore_around_nose,yellow_crust_ooze,prognosis
2453,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,1,Impetigo
4294,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Varicose veins
2083,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Chronic cholestasis
4360,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Jaundice
4739,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Alcoholic hepatitis


In [124]:
df.shape

(4920, 133)

In [125]:
X=df.iloc[:,:-1]
y=df.iloc[:,-1]

In [126]:
le=LabelEncoder()
y_encoded=le.fit_transform(y)

In [127]:
X_train, X_test, y_train, y_test=train_test_split(X, y_encoded, test_size=0.2, random_state=42)

In [128]:
# Initialize the models
rf_clf = RandomForestClassifier(n_estimators=100, random_state=42)
svm_clf = SVC(kernel='linear', probability=True, random_state=42)
nb_clf = MultinomialNB()
lr_clf = LogisticRegression()

# Combine the models using a majority voting strategy
ensemble_clf = VotingClassifier(estimators=[('rf', rf_clf), ('svm', svm_clf), ('nb', nb_clf), ('lr', lr_clf)], voting='soft')


In [129]:
# Train the ensemble model
ensemble_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('rf', RandomForestClassifier(random_state=42)),
                             ('svm',
                              SVC(kernel='linear', probability=True,
                                  random_state=42)),
                             ('nb', MultinomialNB()),
                             ('lr', LogisticRegression())],
                 voting='soft')

In [130]:
y_pred=ensemble_clf.predict(X_test)

In [131]:
acc=accuracy_score(y_test, y_pred)
print(f"Accuracy: {acc}")

conf_mat=confusion_matrix(y_test, y_pred)
print(f"{conf_mat}")

Accuracy: 1.0
[[18  0  0 ...  0  0  0]
 [ 0 30  0 ...  0  0  0]
 [ 0  0 24 ...  0  0  0]
 ...
 [ 0  0  0 ... 26  0  0]
 [ 0  0  0 ...  0 22  0]
 [ 0  0  0 ...  0  0 34]]


In [132]:
pickle.dump(ensemble_clf, open('model.pkl', 'wb'))